In [ ]:
# Cell 1: Imports and Setup

import requests
import os
import time
import datetime as dt
from datetime import timezone, timedelta
import base64
import json
from pathlib import Path
from cryptography.hazmat.primitives import serialization, hashes
from cryptography.hazmat.primitives.asymmetric import padding, rsa
import pandas as pd
from tqdm.notebook import tqdm
from dotenv import load_dotenv
import logging
import re
import glob # For finding files

# --- Logging Setup ---
logger_name = f"kalshi_NTM_fetch_{dt.datetime.now().strftime('%Y%m%d_%H%M%S')}"
logger = logging.getLogger(logger_name)
if not logger.handlers:
    logger.setLevel(logging.INFO)
    formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(name)s.%(funcName)s:%(lineno)d - %(message)s')
    ch = logging.StreamHandler()
    ch.setFormatter(formatter)
    logger.addHandler(ch)
    # Optional: Add file handler for persistent logs
    # log_file_path = f"{logger_name}.log"
    # fh = logging.FileHandler(log_file_path)
    # fh.setFormatter(formatter)
    # logger.addHandler(fh)
else:
    logger.setLevel(logging.INFO)

# --- Configuration ---
load_dotenv()
KALSHI_API_KEY_ID = os.getenv("KALSHI_API_KEY_ID")
KALSHI_PRIVATE_KEY_PATH = os.getenv("KALSHI_PRIVATE_KEY_PATH")
KALSHI_BASE_URL = ""

IS_DEMO_MODE = os.getenv("KALSHI_DEMO_MODE", "false").lower() == "true"
if IS_DEMO_MODE:
    logger.info("KALSHI: Running in DEMO mode.")
    KALSHI_BASE_URL = "https://demo-api.kalshi.co"
    KALSHI_DEMO_API_KEY_ID = os.getenv("KALSHI_DEMO_API_KEY_ID")
    KALSHI_DEMO_PRIVATE_KEY_PATH = os.getenv("KALSHI_DEMO_PRIVATE_KEY_PATH")
    if KALSHI_DEMO_API_KEY_ID: KALSHI_API_KEY_ID = KALSHI_DEMO_API_KEY_ID
    if KALSHI_DEMO_PRIVATE_KEY_PATH: KALSHI_PRIVATE_KEY_PATH = KALSHI_DEMO_PRIVATE_KEY_PATH
else:
    logger.info("KALSHI: Running in PRODUCTION mode.")
    KALSHI_BASE_URL = "https://api.elections.kalshi.com"

# --- Auth Functions (copied from discover.ipynb) ---
private_key_global = None

def load_private_key(file_path: str) -> rsa.RSAPrivateKey | None:
    global private_key_global
    if not file_path:
        logger.error("Private key file path is not provided.")
        return None
    try:
        expanded_path = Path(file_path).expanduser().resolve()
        if not expanded_path.exists():
            logger.error(f"Private key file does not exist at resolved path: {expanded_path}")
            private_key_global = None
            return None
        with open(expanded_path, "rb") as key_file:
            private_key_global = serialization.load_pem_private_key(key_file.read(), password=None)
        logger.info(f"Private key loaded successfully from {expanded_path}")
        return private_key_global
    except FileNotFoundError: # Should be caught by exists() check, but good to have
        logger.error(f"Private key file not found: {file_path}")
        private_key_global = None
        return None
    except Exception as e:
        logger.error(f"Error loading private key from {file_path}: {e}")
        private_key_global = None
        return None

def sign_pss_text(private_key: rsa.RSAPrivateKey, text: str) -> str | None:
    # ... (same as your Cell 1) ...
    if not private_key:
        logger.error("Private key not available for signing.")
        return None
    message = text.encode('utf-8')
    try:
        signature = private_key.sign(
            message, padding.PSS(mgf=padding.MGF1(hashes.SHA256()), salt_length=padding.PSS.DIGEST_LENGTH),
            hashes.SHA256()
        )
        return base64.b64encode(signature).decode('utf-8')
    except Exception as e:
        logger.error(f"Error during signing: {e}")
        return None

def get_kalshi_auth_headers(method: str, path: str) -> dict | None:
    # ... (same as your Cell 1) ...
    if not private_key_global:
        logger.error("Global private_key_global not loaded. Cannot create auth headers.")
        return None
    if not KALSHI_API_KEY_ID:
        logger.error("Global KALSHI_API_KEY_ID not set. Cannot create auth headers.")
        return None
        
    timestamp_ms_str = str(int(time.time() * 1000))
    if not path.startswith('/'):
        path = '/' + path
    message_to_sign = timestamp_ms_str + method.upper() + path
    signature = sign_pss_text(private_key_global, message_to_sign)
    if signature is None: return None
    
    return {
        'accept': 'application/json',
        'KALSHI-ACCESS-KEY': KALSHI_API_KEY_ID,
        'KALSHI-ACCESS-SIGNATURE': signature,
        'KALSHI-ACCESS-TIMESTAMP': timestamp_ms_str
    }

# --- Initialize private key ---
if not (KALSHI_API_KEY_ID and KALSHI_PRIVATE_KEY_PATH):
    logger.critical("CRITICAL: KALSHI_API_KEY_ID or KALSHI_PRIVATE_KEY_PATH not found.")
else:
    load_private_key(KALSHI_PRIVATE_KEY_PATH)

if private_key_global:
    logger.info("Kalshi client setup complete. Private key loaded.")
else:
    logger.error("Kalshi client setup failed: Private key could not be loaded. API calls will fail.")

In [2]:
# Cell 2: Kalshi API Request Function

def kalshi_api_get_request(endpoint_path: str, params: dict = None, timeout: int = 20) -> dict | None:
    """
    Makes an authenticated GET request to the Kalshi API.
    Allows overriding timeout.
    """
    if not private_key_global:
        logger.error("Private key is not loaded. Cannot make API request.")
        return None
    if not KALSHI_BASE_URL:
        logger.error("KALSHI_BASE_URL is not set. Cannot make API request.")
        return None

    if not endpoint_path.startswith('/'):
        endpoint_path = '/' + endpoint_path
        
    full_url = f"{KALSHI_BASE_URL}{endpoint_path}"
    auth_headers = get_kalshi_auth_headers("GET", endpoint_path)
    if not auth_headers:
        logger.error(f"Failed to generate authentication headers for path: {endpoint_path}")
        return None

    try:
        # logger.info(f"Making GET request to: {full_url} with params: {params}") # Can be too verbose
        response = requests.get(full_url, headers=auth_headers, params=params, timeout=timeout)
        response.raise_for_status()
        return response.json()
    except requests.exceptions.HTTPError as http_err:
        logger.error(f"HTTP error: {http_err} | Status: {response.status_code if 'response' in locals() else 'N/A'} | URL: {full_url} | Response: {response.text if 'response' in locals() else 'N/A'}")
    except requests.exceptions.ConnectionError as conn_err:
        logger.error(f"Connection error: {conn_err} | URL: {full_url}")
    except requests.exceptions.Timeout as timeout_err:
        logger.error(f"Timeout error: {timeout_err} | URL: {full_url}")
    except requests.exceptions.RequestException as req_err:
        logger.error(f"Request error: {req_err} | URL: {full_url}")
    except json.JSONDecodeError:
        logger.error(f"JSON decode error | URL: {full_url} | Response: {response.text[:500] if 'response' in locals() else 'N/A'}...")
    return None

In [ ]:
# Cell 3: Configuration & Utility Functions for NTM Download

# --- Date Range for Historical Kalshi Data ---
# This date range is NOT used by the current Cell 4 logic, which processes a predefined CSV.
# Kept here for context or if you switch back to date-range based discovery.
START_DATE_HISTORICAL_STR = "2024-04-01"
YESTERDAY = (dt.datetime.now(timezone.utc) - timedelta(days=1)).strftime("%Y-%m-%d")
END_DATE_HISTORICAL_STR = YESTERDAY

# --- NTM (Near-The-Money) Filter Configuration ---
NTM_PERCENTAGE_THRESHOLD = 0.025 # e.g., 7.5% around BTC price.
logger.info(f"Using NTM Percentage Threshold: {NTM_PERCENTAGE_THRESHOLD*100:.2f}%")

# --- Kalshi Event Title Filter (Not directly used if processing a pre-filtered ticker list, but good to have) ---
EVENT_TITLE_REGEX_PATTERN = r"Bitcoin price .*? at \d{1,2}(?:am|pm)? EDT\?"
EVENT_TITLE_REGEX = re.compile(EVENT_TITLE_REGEX_PATTERN, re.IGNORECASE)
# logger.info(f"Using event title regex pattern: '{EVENT_TITLE_REGEX_PATTERN}'") # Less relevant for CSV processing

# --- Data Directories ---
BINANCE_DATA_BASE_DIR = Path("./binance_data")
KALSHI_ORGANIZED_NTM_DATA_DIR = Path("./kalshi_data") # Changed as per your request
KALSHI_ORGANIZED_NTM_DATA_DIR.mkdir(parents=True, exist_ok=True)
logger.info(f"NTM Kalshi data will be saved under: {KALSHI_ORGANIZED_NTM_DATA_DIR}")

# --- Binance Data Cache & Loading ---
_binance_day_data_cache = {}

def clear_binance_day_cache():
    global _binance_day_data_cache
    _binance_day_data_cache = {}

def load_binance_day_data(date_str_yyyy_mm_dd: str) -> pd.DataFrame | None:
    global _binance_day_data_cache
    if date_str_yyyy_mm_dd in _binance_day_data_cache:
        return _binance_day_data_cache[date_str_yyyy_mm_dd]
    filename_base = f"BTCUSDT-1m-{date_str_yyyy_mm_dd}"
    filepath = BINANCE_DATA_BASE_DIR / filename_base / f"{filename_base}.csv"
    if not filepath.exists():
        logger.warning(f"Binance data file not found for {date_str_yyyy_mm_dd} at {filepath}")
        _binance_day_data_cache[date_str_yyyy_mm_dd] = None
        return None
    try:
        column_names = ["open_time_raw", "open", "high", "low", "close", "volume",
                        "close_time_ms", "quote_asset_volume", "number_of_trades",
                        "taker_buy_base_asset_volume", "taker_buy_quote_asset_volume", "ignore"]
        df = pd.read_csv(filepath, header=None, names=column_names)
        if df.empty:
            logger.warning(f"Binance data file {filepath} is empty.")
            _binance_day_data_cache[date_str_yyyy_mm_dd] = None
            return None
        df['timestamp_s'] = df['open_time_raw'] // 1_000_000
        df.set_index('timestamp_s', inplace=True)
        df['close'] = pd.to_numeric(df['close'])
        _binance_day_data_cache[date_str_yyyy_mm_dd] = df
        return df
    except Exception as e:
        logger.error(f"Error loading Binance data from {filepath}: {e}")
        _binance_day_data_cache[date_str_yyyy_mm_dd] = None
        return None

def get_btc_price_at_timestamp(target_timestamp_s: int) -> float | None:
    target_dt = dt.datetime.fromtimestamp(target_timestamp_s, tz=timezone.utc)
    date_str_needed = target_dt.strftime("%Y-%m-%d")
    binance_df = load_binance_day_data(date_str_needed)
    if binance_df is None or binance_df.empty: return None
    try:
        idx_pos = binance_df.index.searchsorted(target_timestamp_s, side='right')
        if idx_pos == 0:
            if target_dt.hour == 0 and target_dt.minute < 5:
                prev_date_dt = target_dt - timedelta(days=1)
                prev_date_str = prev_date_dt.strftime("%Y-%m-%d")
                binance_df_prev = load_binance_day_data(prev_date_str)
                if binance_df_prev is not None and not binance_df_prev.empty:
                    idx_pos_prev = binance_df_prev.index.searchsorted(target_timestamp_s, side='right')
                    if idx_pos_prev > 0: return binance_df_prev.iloc[idx_pos_prev - 1]['close']
            return None
        return float(binance_df.iloc[idx_pos - 1]['close'])
    except Exception: return None # Simplified error handling for brevity

# --- Kalshi Ticker Parsing & Date Utilities ---
def get_event_resolution_details(ticker_string: str | None):
    if not ticker_string: return None
    event_match = re.match(r"^(.*?)-(\d{2}[A-Z]{3}\d{2})(\d{2})$", ticker_string)
    market_match = re.match(r"^(.*?)-(\d{2}[A-Z]{3}\d{2})(\d{2})-(T(\d+\.?\d*))$", ticker_string)
    match_to_use = market_match if market_match else event_match
    if not match_to_use: return None
    groups = match_to_use.groups()
    series, date_str_yymmmdd, hour_str_edt = groups[0], groups[1], groups[2]
    strike_price = float(groups[4]) if market_match and len(groups) > 4 and groups[4] else None
    try:
        year_int = 2000 + int(date_str_yymmmdd[:2])
        month_str = date_str_yymmmdd[2:5].upper()
        day_int = int(date_str_yymmmdd[5:])
        month_map = {'JAN': 1, 'FEB': 2, 'MAR': 3, 'APR': 4, 'MAY': 5, 'JUN': 6,
                     'JUL': 7, 'AUG': 8, 'SEP': 9, 'OCT': 10, 'NOV': 11, 'DEC': 12}
        month_int = month_map[month_str]
        hour_edt_int = int(hour_str_edt)
        event_resolution_dt_naive_edt = dt.datetime(year_int, month_int, day_int, hour_edt_int, 0, 0)
        utc_offset_hours = 4
        event_resolution_dt_utc_aware = event_resolution_dt_naive_edt.replace(tzinfo=timezone(timedelta(hours=-utc_offset_hours)))
        event_resolution_dt_utc = event_resolution_dt_utc_aware.astimezone(timezone.utc)
        return {"series": series, "date_str_yymmmdd": date_str_yymmmdd, "hour_str_edt": hour_str_edt,
                "strike_price": strike_price, "event_resolution_dt_utc": event_resolution_dt_utc}
    except Exception as e:
        logger.error(f"Error parsing ticker {ticker_string} in get_event_resolution_details: {e}")
        return None

def parse_iso_to_unix_timestamp(date_string: str | None) -> int | None:
    if not date_string: return None
    try:
        if date_string.endswith('Z'): dt_obj = dt.datetime.fromisoformat(date_string.replace('Z', '+00:00'))
        else:
            dt_obj = dt.datetime.fromisoformat(date_string)
            if dt_obj.tzinfo is None: dt_obj = dt_obj.replace(tzinfo=timezone.utc)
        return int(dt_obj.timestamp())
    except Exception as e:
        logger.error(f"Error parsing date string '{date_string}' to Unix ts: {e}")
        return None

# --- Candlestick Fetching Function ---
_first_candle_structure_logged_session = False
def fetch_candlesticks_for_market(market_ticker: str, series_ticker: str, start_ts_s: int,
                                  end_ts_s: int, period_minutes: int = 1, api_delay_seconds: float = 0.6
                                 ) -> pd.DataFrame | None:
    global _first_candle_structure_logged_session
    all_candlesticks_data = []
    if not all([market_ticker, series_ticker, isinstance(start_ts_s, int), isinstance(end_ts_s, int), period_minutes > 0]):
        logger.error(f"Invalid params for fetch_candlesticks_for_market for {market_ticker}.")
        return None
    if start_ts_s >= end_ts_s: return pd.DataFrame()
    current_start_ts = start_ts_s
    max_periods_per_request = 4900
    with tqdm(total=(end_ts_s - start_ts_s) // (period_minutes * 60) + 1, desc=f"Candles: {market_ticker[:20]}...", leave=False, unit="cndl") as pbar:
        while current_start_ts < end_ts_s:
            chunk_max_duration_seconds = (max_periods_per_request - 1) * period_minutes * 60
            chunk_end_ts = min(end_ts_s, current_start_ts + chunk_max_duration_seconds)
            if chunk_end_ts <= current_start_ts: break
            api_path = f"/trade-api/v2/series/{series_ticker}/markets/{market_ticker}/candlesticks"
            params = {"start_ts": current_start_ts, "end_ts": chunk_end_ts, "period_interval": period_minutes}
            response_json = kalshi_api_get_request(api_path, params, timeout=30)
            time.sleep(api_delay_seconds) # Moved delay here to ensure it happens after every API call

            if response_json and "candlesticks" in response_json:
                candlesticks_from_api = response_json["candlesticks"]
                if not _first_candle_structure_logged_session and candlesticks_from_api:
                    logger.info(f"First candlestick structure: {json.dumps(candlesticks_from_api[0], indent=2)}")
                    _first_candle_structure_logged_session = True
                chunk_data = []
                if candlesticks_from_api:
                    for candle in candlesticks_from_api:
                        ts = candle.get("end_period_ts")
                        if ts is None or ts > chunk_end_ts or ts < current_start_ts: continue
                        trade_price_info = candle.get("price", {}) or {}
                        yes_bid_info = candle.get("yes_bid", {}) or {}
                        yes_ask_info = candle.get("yes_ask", {}) or {}
                        chunk_data.append({
                            "market_ticker": market_ticker, "series_ticker": series_ticker, "timestamp_s": ts,
                            "datetime_utc": dt.datetime.fromtimestamp(ts, tz=timezone.utc).isoformat(),
                            "trade_open_cents": trade_price_info.get("open"), "trade_high_cents": trade_price_info.get("high"),
                            "trade_low_cents": trade_price_info.get("low"), "trade_close_cents": trade_price_info.get("close"),
                            "yes_bid_open_cents": yes_bid_info.get("open"), "yes_bid_high_cents": yes_bid_info.get("high"),
                            "yes_bid_low_cents": yes_bid_info.get("low"), "yes_bid_close_cents": yes_bid_info.get("close"),
                            "yes_ask_open_cents": yes_ask_info.get("open"), "yes_ask_high_cents": yes_ask_info.get("high"),
                            "yes_ask_low_cents": yes_ask_info.get("low"), "yes_ask_close_cents": yes_ask_info.get("close"),
                            "volume": candle.get("volume"), "open_interest": candle.get("open_interest")
                        })
                    all_candlesticks_data.extend(chunk_data)
                    pbar.update(len(chunk_data) if chunk_data else max(1, (chunk_end_ts - current_start_ts) // (period_minutes*60)))
                    last_ts_in_chunk = candlesticks_from_api[-1].get("end_period_ts", chunk_end_ts) if candlesticks_from_api else chunk_end_ts
                    current_start_ts = last_ts_in_chunk + (period_minutes * 60)
                else: # No candlesticks in API response for this chunk
                    pbar.update(max(1, (chunk_end_ts - current_start_ts) // (period_minutes*60)))
                    current_start_ts = chunk_end_ts + (period_minutes * 60)
            else: # API request failed or bad response
                logger.warning(f"No candlestick data or API error for {market_ticker} chunk: {params}")
                pbar.update(max(1, (chunk_end_ts - current_start_ts) // (period_minutes*60)))
                current_start_ts = chunk_end_ts + (period_minutes * 60)
                time.sleep(api_delay_seconds * 2) # Longer delay on error
    if not all_candlesticks_data: return pd.DataFrame()
    df = pd.DataFrame(all_candlesticks_data)
    df.sort_values("timestamp_s", inplace=True)
    df.drop_duplicates(subset=["timestamp_s"], keep="first", inplace=True)
    return df

logger.info("Cell 3: Configurations and Utility Functions defined.")

In [ ]:
# Cell 4: Read Master Ticker List, Filter for NTM, and Prepare for Download

# --- Configuration for this Cell ---
# Path to your CSV file containing all historical market tickers
# Find the latest one if multiple exist, or specify the exact path.
try:
    # Assuming TARGET_SERIES_TICKER is defined in Cell 3 (e.g., "KXBTCD")
    if 'TARGET_SERIES_TICKER' not in globals(): # Define if not from previous cell
        TARGET_SERIES_TICKER = "KXBTCD" # Default if not defined by Cell 3 (though it should be)
        logger.info(f"TARGET_SERIES_TICKER was not defined in Cell 3, set to default: {TARGET_SERIES_TICKER}")

    list_of_ticker_files = glob.glob(f"kalshi_btc_hourly_market_tickers_{TARGET_SERIES_TICKER}_*.csv")
    if not list_of_ticker_files:
        raise FileNotFoundError(f"No master ticker CSV files found matching pattern: kalshi_btc_hourly_market_tickers_{TARGET_SERIES_TICKER}_*.csv. Please ensure Cell 3 of discover.ipynb (or equivalent) has been run to generate this file.")
    MASTER_TICKER_CSV_PATH = max(list_of_ticker_files, key=os.path.getctime)
    logger.info(f"Using master ticker list from: {MASTER_TICKER_CSV_PATH}")
except FileNotFoundError as e:
    logger.critical(str(e))
    raise
except NameError as e: # Catches if TARGET_SERIES_TICKER isn't defined
    logger.critical(f"A required variable (likely TARGET_SERIES_TICKER) for finding the ticker CSV is not defined: {e}. Ensure Cell 3 has run or define it.")
    raise


# API call delay for fetching market details
API_DELAY_MARKET_DETAILS = 0.05  # seconds - can be small if calls are infrequent per second overall
                                # Original was 0.5. If you hit rate limits, increase this.

# --- Lists to store NTM markets and outcomes ---
ntm_markets_to_download_info = [] # List of tuples: (market_ticker, series_ticker, open_ts, close_ts, event_date_dir, event_hour_dir)
ntm_market_outcomes_data = []   # List of dicts for outcomes CSV

# --- Load the master list of market tickers ---
try:
    master_tickers_df = pd.read_csv(MASTER_TICKER_CSV_PATH)
    if 'market_ticker' not in master_tickers_df.columns:
        logger.critical(f"'market_ticker' column not found in {MASTER_TICKER_CSV_PATH}.")
        raise ValueError("CSV must contain 'market_ticker' column")
    all_market_tickers_from_csv = master_tickers_df['market_ticker'].dropna().unique().tolist()
    logger.info(f"Loaded {len(all_market_tickers_from_csv)} unique market tickers from {MASTER_TICKER_CSV_PATH}.")
except Exception as e:
    logger.critical(f"Error loading or processing master ticker CSV {MASTER_TICKER_CSV_PATH}: {e}")
    raise

# --- Helper function to fetch specific market details (open/close times, result) ---
# This function now RELIES on get_event_resolution_details being defined in Cell 3.
def fetch_specific_market_details_for_ntm(market_ticker: str) -> dict | None:
    """Fetches open_time, close_time, and result for a specific market_ticker."""
    api_path = f"/trade-api/v2/markets/{market_ticker}"
    response_data = kalshi_api_get_request(api_path, timeout=15)
    time.sleep(API_DELAY_MARKET_DETAILS)

    if response_data and "market" in response_data:
        market_data_from_api = response_data["market"]
        
        # get_event_resolution_details is now expected to be globally available from Cell 3
        parsed_ticker_for_datetime_and_series = get_event_resolution_details(market_ticker)
        if not parsed_ticker_for_datetime_and_series:
            logger.warning(f"Could not parse {market_ticker} with get_event_resolution_details. Skipping.")
            return None

        derived_series_ticker = parsed_ticker_for_datetime_and_series.get("series")

        details = {
            "market_ticker": market_ticker,
            "series_ticker": derived_series_ticker,
            "open_ts": parse_iso_to_unix_timestamp(market_data_from_api.get("open_time")),
            "close_ts": parse_iso_to_unix_timestamp(market_data_from_api.get("close_time")),
            "result": market_data_from_api.get("result"),
            "status": market_data_from_api.get("status"),
            "strike_price": parsed_ticker_for_datetime_and_series.get("strike_price"),
            "event_resolution_dt_utc": parsed_ticker_for_datetime_and_series.get("event_resolution_dt_utc"),
            "date_str_yymmmdd_for_dir": parsed_ticker_for_datetime_and_series.get("date_str_yymmmdd"),
            "hour_str_edt_close_for_dir": parsed_ticker_for_datetime_and_series.get("hour_str_edt")
        }
        
        required_keys = ["open_ts", "close_ts", "series_ticker", "event_resolution_dt_utc", 
                         "date_str_yymmmdd_for_dir", "hour_str_edt_close_for_dir"]
        missing_fields = [k for k in required_keys if details.get(k) is None]
        if details["strike_price"] is None: missing_fields.append("strike_price")
            
        if missing_fields:
            logger.warning(f"Incomplete market details for {market_ticker}. Missing or None fields: {missing_fields}. API Data: {str(market_data_from_api)[:200]}. Parsed: {details}. Skipping.")
            return None
        return details
    else:
        logger.error(f"Failed to fetch details for market {market_ticker}. Response: {str(response_data)[:200]}")
        return None

# --- Iterate through each market ticker from the CSV ---
if 'NTM_PERCENTAGE_THRESHOLD' not in globals(): # From Cell 3
    logger.critical("NTM_PERCENTAGE_THRESHOLD not defined. Please run Cell 3.")
    raise NameError("NTM_PERCENTAGE_THRESHOLD not defined")

logger.info(f"Processing {len(all_market_tickers_from_csv)} market tickers for NTM status (NTM Threshold: {NTM_PERCENTAGE_THRESHOLD*100:.2f}%)...")

# TEST_LIMIT_TICKERS = 50 # Keep small for testing
TEST_LIMIT_TICKERS = None 

processed_ticker_count = 0
for market_ticker_from_csv in tqdm(all_market_tickers_from_csv, desc="Filtering NTM Markets"):
    if TEST_LIMIT_TICKERS is not None and processed_ticker_count >= TEST_LIMIT_TICKERS:
        logger.info(f"Reached test limit of {TEST_LIMIT_TICKERS} tickers. Stopping NTM identification.")
        break
    processed_ticker_count += 1

    market_details = fetch_specific_market_details_for_ntm(market_ticker_from_csv)
    if not market_details:
        continue

    event_resolution_dt_utc = market_details["event_resolution_dt_utc"]
    market_strike_price = market_details["strike_price"]
    reference_btc_time_ts = int(event_resolution_dt_utc.timestamp()) - 3600 # 1hr before event resolution
    
    # get_btc_price_at_timestamp is defined in Cell 3
    reference_btc_price = get_btc_price_at_timestamp(reference_btc_time_ts)

    if reference_btc_price is None:
        ref_time_str = dt.datetime.fromtimestamp(reference_btc_time_ts, timezone.utc).isoformat()
        logger.warning(f"No Binance BTC price near {ref_time_str} for NTM check of {market_ticker_from_csv}. Skipping.")
        continue

    is_ntm = False
    if reference_btc_price > 0:
        price_diff_percentage = abs(reference_btc_price - market_strike_price) / reference_btc_price
        if price_diff_percentage <= NTM_PERCENTAGE_THRESHOLD:
            is_ntm = True
    
    if is_ntm:
        if market_details["status"] not in ["closed", "settled", "finalized"]:
            # logger.info(f"  NTM market {market_ticker_from_csv} status '{market_details['status']}', not fully historical. Skipping.")
            continue # Skip if not yet settled, as outcome might not be final
            
        logger.info(f"  NTM MATCH: {market_ticker_from_csv} (Strike: {market_strike_price:,.2f}, RefBTC: {reference_btc_price:,.2f}). Queuing.")
        ntm_markets_to_download_info.append((
            market_details["market_ticker"], market_details["series_ticker"],
            market_details["open_ts"], market_details["close_ts"],
            market_details["date_str_yymmmdd_for_dir"], market_details["hour_str_edt_close_for_dir"]
        ))
        ntm_market_outcomes_data.append({
            "market_ticker": market_details["market_ticker"], "result": market_details["result"],
            "event_resolution_time_iso": event_resolution_dt_utc.isoformat(),
            "reference_btc_price_for_ntm": reference_btc_price, "kalshi_strike_price": market_strike_price,
            "market_open_time_iso": dt.datetime.fromtimestamp(market_details["open_ts"], timezone.utc).isoformat(),
            "market_close_time_iso": dt.datetime.fromtimestamp(market_details["close_ts"], timezone.utc).isoformat(),
            "event_ticker_parent": market_ticker_from_csv.split('-T')[0]
        })

logger.info(f"\n--- NTM Market Identification from CSV Complete ---")
logger.info(f"Processed {processed_ticker_count} tickers from CSV.")
logger.info(f"Found {len(ntm_markets_to_download_info)} NTM markets to download.")
logger.info(f"Collected {len(ntm_market_outcomes_data)} NTM market outcomes.")

if ntm_markets_to_download_info:
    print(f"\nSample of NTM markets queued for download (first 5):")
    for item in ntm_markets_to_download_info[:5]: print(f"  {item}")
else: print("\nNo NTM markets identified for download from the CSV.")

if ntm_market_outcomes_data:
    df_ntm_outcomes_preview = pd.DataFrame(ntm_market_outcomes_data)
    if not df_ntm_outcomes_preview.empty:
        print("\nPreview of NTM Outcomes DataFrame (first 5 rows):")
        print(df_ntm_outcomes_preview.head().to_string())
else: print("\nNo NTM market outcomes data collected.")

In [ ]:
# Cell 5: Downloading Candlestick Data for Filtered NTM Markets & Saving Outcomes

total_ntm_markets = len(ntm_markets_to_download_info)
logger.info(f"\n--- Starting Candlestick Download for {total_ntm_markets} NTM Markets (identified from CSV) ---")

downloaded_count = 0
skipped_already_exists_count = 0
failed_download_count = 0

# Define the path for the consolidated outcomes CSV
outcomes_csv_filename = f"kalshi_btc_hourly_NTM_filtered_market_outcomes_{dt.datetime.now().strftime('%Y%m%d_%H%M%S')}.csv"

# Ensure KALSHI_ORGANIZED_NTM_DATA_DIR is defined from Cell 3
if 'KALSHI_ORGANIZED_NTM_DATA_DIR' not in globals():
    logger.critical("KALSHI_ORGANIZED_NTM_DATA_DIR not defined. Please run Cell 3.")
    raise NameError("KALSHI_ORGANIZED_NTM_DATA_DIR not defined.")
outcomes_csv_filepath = KALSHI_ORGANIZED_NTM_DATA_DIR / outcomes_csv_filename # Save INSIDE kalshi_data

# Use tqdm for the outer loop of markets
for market_info_tuple in tqdm(ntm_markets_to_download_info, desc="Downloading NTM Market Candles"):
    market_ticker, series_ticker, market_open_ts, market_close_ts, event_date_dir_str, event_hour_dir_str = market_info_tuple
    
    target_day_dir = KALSHI_ORGANIZED_NTM_DATA_DIR / event_date_dir_str
    target_hour_dir = target_day_dir / event_hour_dir_str.zfill(2)
    target_hour_dir.mkdir(parents=True, exist_ok=True)
    market_csv_filepath = target_hour_dir / f"{market_ticker}.csv"

    if market_csv_filepath.exists() and market_csv_filepath.stat().st_size > 0:
        skipped_already_exists_count += 1
        continue
        
    # fetch_candlesticks_for_market is defined in Cell 3
    candlesticks_df = fetch_candlesticks_for_market(
        market_ticker=market_ticker, series_ticker=series_ticker, 
        start_ts_s=market_open_ts, end_ts_s=market_close_ts, period_minutes=1 
    )

    if candlesticks_df is not None and not candlesticks_df.empty:
        try:
            candlesticks_df.to_csv(market_csv_filepath, index=False)
            downloaded_count += 1
        except Exception as e:
            logger.error(f"Error saving data for {market_ticker} to {market_csv_filepath}: {e}")
            failed_download_count += 1
    elif candlesticks_df is not None and candlesticks_df.empty:
        logger.info(f"No candle data from API for NTM market {market_ticker}. No CSV created.")
        failed_download_count +=1 
    else: # candlesticks_df is None (API error)
        logger.warning(f"Failed to fetch candle data for {market_ticker}. No CSV created.")
        failed_download_count += 1
    
logger.info("\n--- NTM Candlestick Download Complete (from CSV list) ---")
logger.info(f"Total NTM markets for download: {total_ntm_markets}")
logger.info(f"Successfully downloaded new data for: {downloaded_count} markets")
logger.info(f"Skipped (already existed): {skipped_already_exists_count} markets")
logger.info(f"Failed to download/save (or empty from API): {failed_download_count} markets")
logger.info(f"Data saved in: {KALSHI_ORGANIZED_NTM_DATA_DIR}")

# --- Save NTM Market Outcomes ---
if ntm_market_outcomes_data:
    outcomes_df = pd.DataFrame(ntm_market_outcomes_data)
    try:
        outcomes_df.to_csv(outcomes_csv_filepath, index=False)
        logger.info(f"Successfully saved {len(outcomes_df)} NTM market outcomes to: {outcomes_csv_filepath}")
        print(f"\nNTM Market outcomes saved to: {outcomes_csv_filepath}")
        if not outcomes_df.empty:
            print("Sample of outcomes data (first 5 rows):")
            print(outcomes_df.head().to_string())
    except Exception as e:
        logger.error(f"Error saving NTM market outcomes to {outcomes_csv_filepath}: {e}")
else:
    logger.info("No NTM market outcomes data to save.")

print(f"\n\nAll NTM data processing finished. Check logs and the directory: {KALSHI_ORGANIZED_NTM_DATA_DIR}")
if ntm_market_outcomes_data:
    print(f"Consolidated outcomes CSV at: {outcomes_csv_filepath}")